# Vectorization

Big Picture
- What is vectorization?
- When is it useful?
- What can the compiler do?
- How can you do it manually?


First lets look at an example problem

# Problem 1

Given a list of data count the pairs of a given byte

```c++
uint64_t count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  // we will want to compare each short to the pair of target bytes
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size; i++) {
    // boolean automaitcally converts to an integer (true = 1, false = 0)
    total += (data[i] == check);
  }
  return total;
}

```

We will be doing all of our experiments with 1 GiB of data

We compile and run this, and it takes 1 second to run

Let us take a look at what it is doing

```
Dump of assembler code for function count_pairs:
ex1a.c:
7	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
   0x0000000000400780 <+0>:	55	push   %rbp
   0x0000000000400781 <+1>:	48 89 e5	mov    %rsp,%rbp
   0x0000000000400784 <+4>:	48 89 7d f8	mov    %rdi,-0x8(%rbp)
   0x0000000000400788 <+8>:	48 89 75 f0	mov    %rsi,-0x10(%rbp)
   0x000000000040078c <+12>:	88 55 ef	mov    %dl,-0x11(%rbp)

8	  uint64_t total = 0;
   0x000000000040078f <+15>:	48 c7 45 e0 00 00 00 00	movq   $0x0,-0x20(%rbp)

9	  uint16_t check = target | (target << 8U);
   0x0000000000400797 <+23>:	0f b6 45 ef	movzbl -0x11(%rbp),%eax
   0x000000000040079b <+27>:	0f b6 4d ef	movzbl -0x11(%rbp),%ecx
   0x000000000040079f <+31>:	c1 e1 08	shl    $0x8,%ecx
   0x00000000004007a2 <+34>:	09 c8	or     %ecx,%eax
   0x00000000004007a4 <+36>:	66 89 45 de	mov    %ax,-0x22(%rbp)

10	  for (uint64_t i = 0; i < size; i++) {
   0x00000000004007a8 <+40>:	48 c7 45 d0 00 00 00 00	movq   $0x0,-0x30(%rbp)
   0x00000000004007b0 <+48>:	48 8b 45 d0	mov    -0x30(%rbp),%rax
   0x00000000004007b4 <+52>:	48 3b 45 f0	cmp    -0x10(%rbp),%rax
   0x00000000004007b8 <+56>:	0f 83 3c 00 00 00	jae    0x4007fa <count_pairs+122>

11	    total += (data[i] == check);
   0x00000000004007be <+62>:	48 8b 45 f8	mov    -0x8(%rbp),%rax
   0x00000000004007c2 <+66>:	48 8b 4d d0	mov    -0x30(%rbp),%rcx
   0x00000000004007c6 <+70>:	0f b7 14 48	movzwl (%rax,%rcx,2),%edx
   0x00000000004007ca <+74>:	0f b7 75 de	movzwl -0x22(%rbp),%esi
   0x00000000004007ce <+78>:	39 f2	cmp    %esi,%edx
   0x00000000004007d0 <+80>:	40 0f 94 c7	sete   %dil
   0x00000000004007d4 <+84>:	40 80 e7 01	and    $0x1,%dil
   0x00000000004007d8 <+88>:	40 0f b6 d7	movzbl %dil,%edx
   0x00000000004007dc <+92>:	48 63 c2	movslq %edx,%rax
   0x00000000004007df <+95>:	48 03 45 e0	add    -0x20(%rbp),%rax
   0x00000000004007e3 <+99>:	48 89 45 e0	mov    %rax,-0x20(%rbp)

10	  for (uint64_t i = 0; i < size; i++) {
   0x00000000004007e7 <+103>:	48 8b 45 d0	mov    -0x30(%rbp),%rax
   0x00000000004007eb <+107>:	48 05 01 00 00 00	add    $0x1,%rax
   0x00000000004007f1 <+113>:	48 89 45 d0	mov    %rax,-0x30(%rbp)
   0x00000000004007f5 <+117>:	e9 b6 ff ff ff	jmpq   0x4007b0 <count_pairs+48>

12	  }
13	  return total;
   0x00000000004007fa <+122>:	48 8b 45 e0	mov    -0x20(%rbp),%rax
   0x00000000004007fe <+126>:	5d	pop    %rbp
   0x00000000004007ff <+127>:	c3	retq   
End of assembler dump.
```

However, we are running compiled code, and the compiler has the ability to optimize our code for us, so let us tell the compiler to take the same code, but this time optimize it.

Now it only takes 208 milliseconds, which is a speedup of almost 5x.

Lets see what it is doing

```
Dump of assembler code for function count_pairs:
ex1a.c:
7	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
8	  uint64_t total = 0;
9	  uint16_t check = target | (target << 8U);
10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400740 <+0>:	48 85 f6	test   %rsi,%rsi
   0x0000000000400743 <+3>:	74 21	je     0x400766 <count_pairs+38>
   0x0000000000400745 <+5>:	41 89 d2	mov    %edx,%r10d
   0x0000000000400748 <+8>:	41 c1 e2 08	shl    $0x8,%r10d
   0x000000000040074c <+12>:	41 09 d2	or     %edx,%r10d
   0x000000000040074f <+15>:	48 8d 46 ff	lea    -0x1(%rsi),%rax
   0x0000000000400753 <+19>:	41 89 f0	mov    %esi,%r8d
   0x0000000000400756 <+22>:	41 83 e0 03	and    $0x3,%r8d
   0x000000000040075a <+26>:	48 83 f8 03	cmp    $0x3,%rax
   0x000000000040075e <+30>:	73 09	jae    0x400769 <count_pairs+41>
   0x0000000000400760 <+32>:	31 d2	xor    %edx,%edx
   0x0000000000400762 <+34>:	31 c0	xor    %eax,%eax
   0x0000000000400764 <+36>:	eb 56	jmp    0x4007bc <count_pairs+124>
   0x0000000000400766 <+38>:	31 c0	xor    %eax,%eax

12	  }
13	  return total;
   0x0000000000400768 <+40>:	c3	retq   

10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400769 <+41>:	4c 29 c6	sub    %r8,%rsi
   0x000000000040076c <+44>:	31 d2	xor    %edx,%edx
   0x000000000040076e <+46>:	31 c0	xor    %eax,%eax

11	    total += (data[i] == check);
   0x0000000000400770 <+48>:	44 0f b7 0c 57	movzwl (%rdi,%rdx,2),%r9d
   0x0000000000400775 <+53>:	31 c9	xor    %ecx,%ecx
   0x0000000000400777 <+55>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040077a <+58>:	0f 94 c1	sete   %cl
   0x000000000040077d <+61>:	48 01 c1	add    %rax,%rcx
   0x0000000000400780 <+64>:	44 0f b7 4c 57 02	movzwl 0x2(%rdi,%rdx,2),%r9d
   0x0000000000400786 <+70>:	31 c0	xor    %eax,%eax
   0x0000000000400788 <+72>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040078b <+75>:	0f 94 c0	sete   %al
   0x000000000040078e <+78>:	48 01 c8	add    %rcx,%rax
   0x0000000000400791 <+81>:	44 0f b7 4c 57 04	movzwl 0x4(%rdi,%rdx,2),%r9d
   0x0000000000400797 <+87>:	31 c9	xor    %ecx,%ecx
   0x0000000000400799 <+89>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040079c <+92>:	0f 94 c1	sete   %cl
   0x000000000040079f <+95>:	48 01 c1	add    %rax,%rcx
   0x00000000004007a2 <+98>:	44 0f b7 4c 57 06	movzwl 0x6(%rdi,%rdx,2),%r9d
   0x00000000004007a8 <+104>:	31 c0	xor    %eax,%eax
   0x00000000004007aa <+106>:	45 39 ca	cmp    %r9d,%r10d
   0x00000000004007ad <+109>:	0f 94 c0	sete   %al
   0x00000000004007b0 <+112>:	48 01 c8	add    %rcx,%rax

10	  for (uint64_t i = 0; i < size; i++) {
   0x00000000004007b3 <+115>:	48 83 c2 04	add    $0x4,%rdx
   0x00000000004007b7 <+119>:	48 39 d6	cmp    %rdx,%rsi
   0x00000000004007ba <+122>:	75 b4	jne    0x400770 <count_pairs+48>
   0x00000000004007bc <+124>:	4d 85 c0	test   %r8,%r8
   0x00000000004007bf <+127>:	74 28	je     0x4007e9 <count_pairs+169>
   0x00000000004007c1 <+129>:	4c 8d 0c 57	lea    (%rdi,%rdx,2),%r9
   0x00000000004007c5 <+133>:	31 f6	xor    %esi,%esi
   0x00000000004007c7 <+135>:	66 0f 1f 84 00 00 00 00 00	nopw   0x0(%rax,%rax,1)

11	    total += (data[i] == check);
   0x00000000004007d0 <+144>:	41 0f b7 3c 71	movzwl (%r9,%rsi,2),%edi
   0x00000000004007d5 <+149>:	31 d2	xor    %edx,%edx
   0x00000000004007d7 <+151>:	41 39 fa	cmp    %edi,%r10d
   0x00000000004007da <+154>:	0f 94 c2	sete   %dl
   0x00000000004007dd <+157>:	48 01 d0	add    %rdx,%rax

10	  for (uint64_t i = 0; i < size; i++) {
   0x00000000004007e0 <+160>:	48 83 c6 01	add    $0x1,%rsi
   0x00000000004007e4 <+164>:	49 39 f0	cmp    %rsi,%r8
   0x00000000004007e7 <+167>:	75 e7	jne    0x4007d0 <count_pairs+144>

12	  }
13	  return total;
   0x00000000004007e9 <+169>:	c3	retq   
End of assembler dump.

```

Lets take a look at the inner loop
```
   0x0000000000400770 <+48>:	44 0f b7 0c 57	movzwl (%rdi,%rdx,2),%r9d
   0x0000000000400775 <+53>:	31 c9	xor    %ecx,%ecx
   0x0000000000400777 <+55>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040077a <+58>:	0f 94 c1	sete   %cl
   0x000000000040077d <+61>:	48 01 c1	add    %rax,%rcx
   0x0000000000400780 <+64>:	44 0f b7 4c 57 02	movzwl 0x2(%rdi,%rdx,2),%r9d
   0x0000000000400786 <+70>:	31 c0	xor    %eax,%eax
   0x0000000000400788 <+72>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040078b <+75>:	0f 94 c0	sete   %al
   0x000000000040078e <+78>:	48 01 c8	add    %rcx,%rax
   0x0000000000400791 <+81>:	44 0f b7 4c 57 04	movzwl 0x4(%rdi,%rdx,2),%r9d
   0x0000000000400797 <+87>:	31 c9	xor    %ecx,%ecx
   0x0000000000400799 <+89>:	45 39 ca	cmp    %r9d,%r10d
   0x000000000040079c <+92>:	0f 94 c1	sete   %cl
   0x000000000040079f <+95>:	48 01 c1	add    %rax,%rcx
   0x00000000004007a2 <+98>:	44 0f b7 4c 57 06	movzwl 0x6(%rdi,%rdx,2),%r9d
   0x00000000004007a8 <+104>:	31 c0	xor    %eax,%eax
   0x00000000004007aa <+106>:	45 39 ca	cmp    %r9d,%r10d
   0x00000000004007ad <+109>:	0f 94 c0	sete   %al
   0x00000000004007b0 <+112>:	48 01 c8	add    %rcx,%rax
```

We can see it does the same thing 4 times since it unrolled to save on loop control time, we are also alternative between the use of rax and rcx, 

```   
        load the data
   0x0000000000400770 <+48>:	44 0f b7 0c 57	movzwl (%rdi,%rdx,2),%r9d
        clear out the register xoring something with itself results in a 0
   0x0000000000400775 <+53>:	31 c9	xor    %ecx,%ecx
        perform the comparison
   0x0000000000400777 <+55>:	45 39 ca	cmp    %r9d,%r10d
        get the result of the comparison
   0x000000000040077a <+58>:	0f 94 c1	sete   %cl
        add the value to the acculation 
   0x000000000040077d <+61>:	48 01 c1	add    %rax,%rcx
```

![x86 registers](Lec09_source/registers.JPG "X86 registers")


| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 994 |
| O3 | 208 |


Now we are going to add a special flag "-march=native"
This tells the compiler that we will be running the code on the same machine that we are compiling it on.
This allows the compiler to use any special things that this computer has which are non-standard.
Most of the time when we compile code we want it to work across a broad range of machines, but newer hardware comes with more optimizations. 

If you want to see what sort of extra hardware is on your machine you can look with `lscpu` or `cat /proc/cpuinfo`

For example if we look at my laptop 

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              12
On-line CPU(s) list: 0-11
Thread(s) per core:  2
Core(s) per socket:  6
Socket(s):           1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               158
Model name:          Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Stepping:            10
CPU MHz:             2592.009
BogoMIPS:            5184.01
Hypervisor vendor:   Microsoft
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            12288K
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht 
                     syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology cpuid pni pclmulqdq ssse3 fma cx16 pcid 
                     sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single 
                     pti ssbd ibrs ibpb stibp fsgsbase bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt xsaveopt xsavec 
                     xgetbv1 xsaves flush_l1d arch_capabilities
```
Specifically we are looking at the flags

Here is a modern high end intel server

```
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss 
                     ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc 
                     cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid dca 
                     sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault 
                     epb cat_l3 cdp_l3 invpcid_single intel_ppin ssbd mba ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi flexpriority ept 
                     vpid fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid cqm mpx rdt_a avx512f avx512dq rdseed adx smap clflushopt 
                     clwb intel_pt avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves cqm_llc cqm_occup_llc cqm_mbm_total cqm_mbm_local 
                     dtherm ida arat pln pts pku ospke avx512_vnni md_clear flush_l1d arch_capabilities
```

We will mostly be converned with the following types

 - SSE (Streaming SIMD Extensions)
 - AVX (Advanced Vector Extensions)

This introduces 2 terms, SIMD and Vectors

SIMD stands for single instruction multiple data, which means we have a single instruction which processes multiple pieces of data at the same time
AVX is basically just the successor to SIMD

These both allow us to use vectorization 

# What is vectorization

Operations on wide registers

Most registers are between 8 and 64 bits
Operations on these registers happen on the entire register

Vector registers are normally a fair amount bigger 128 - 512 bits

They are normally thought of as a vector of data, where operations happen individually on each item in the vector

![Vector Operation](Lec09_source/vector_op.JPG "Vector Operation")


For example `vpcmpeqw ` is a special instruction which operated on 2 256 bit registers, it treats these registers as having 16, 16-bit objects, compares all 16 of them for equality and then outputs -1 in the same location of the output 256-bit register, if they are equal, and 0 otherwise.

This would allow us to perform the equivalent of 4 iterations of the unrolled loop all at once.

This is still the same code, we are just changing how we compile it 
```c++
uint64_t count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  // we will want to compare each short to the pair of target bytes
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size; i++) {
    total += (data[i] == check);
  }
  return total;
}

```

Compiling with march native gives us over another 2x speedup

| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 994 |
| O3 | 208 |
| march=native | 97 |

Let us take a look at the assembly for this one 

```
Dump of assembler code for function count_pairs:
ex1a.c:
7	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
8	  uint64_t total = 0;
9	  uint16_t check = target | (target << 8U);
10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400740 <+0>:	48 85 f6	test   %rsi,%rsi
   0x0000000000400743 <+3>:	74 1a	je     0x40075f <count_pairs+31>
   0x0000000000400745 <+5>:	41 89 d1	mov    %edx,%r9d
   0x0000000000400748 <+8>:	41 c1 e1 08	shl    $0x8,%r9d
   0x000000000040074c <+12>:	41 09 d1	or     %edx,%r9d
   0x000000000040074f <+15>:	48 83 fe 0f	cmp    $0xf,%rsi
   0x0000000000400753 <+19>:	77 0d	ja     0x400762 <count_pairs+34>
   0x0000000000400755 <+21>:	45 31 d2	xor    %r10d,%r10d
   0x0000000000400758 <+24>:	31 c0	xor    %eax,%eax
   0x000000000040075a <+26>:	e9 d1 01 00 00	jmpq   0x400930 <count_pairs+496>
   0x000000000040075f <+31>:	31 c0	xor    %eax,%eax

12	  }
13	  return total;
   0x0000000000400761 <+33>:	c3	retq   

10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400762 <+34>:	49 89 f2	mov    %rsi,%r10
   0x0000000000400765 <+37>:	49 83 e2 f0	and    $0xfffffffffffffff0,%r10
   0x0000000000400769 <+41>:	c4 c1 79 6e c1	vmovd  %r9d,%xmm0
   0x000000000040076e <+46>:	c4 e2 79 58 c0	vpbroadcastd %xmm0,%xmm0
   0x0000000000400773 <+51>:	49 8d 42 f0	lea    -0x10(%r10),%rax
   0x0000000000400777 <+55>:	48 89 c1	mov    %rax,%rcx
   0x000000000040077a <+58>:	48 c1 e9 04	shr    $0x4,%rcx
   0x000000000040077e <+62>:	48 ff c1	inc    %rcx
   0x0000000000400781 <+65>:	41 89 c8	mov    %ecx,%r8d
   0x0000000000400784 <+68>:	41 83 e0 01	and    $0x1,%r8d
   0x0000000000400788 <+72>:	48 85 c0	test   %rax,%rax
   0x000000000040078b <+75>:	0f 84 bb 01 00 00	je     0x40094c <count_pairs+524>
   0x0000000000400791 <+81>:	4c 29 c1	sub    %r8,%rcx
   0x0000000000400794 <+84>:	c4 41 39 ef c0	vpxor  %xmm8,%xmm8,%xmm8
   0x0000000000400799 <+89>:	31 c0	xor    %eax,%eax
   0x000000000040079b <+91>:	c4 e2 7d 59 2d 74 03 00 00	vpbroadcastq 0x374(%rip),%ymm5        # 0x400b18
   0x00000000004007a4 <+100>:	c4 41 31 ef c9	vpxor  %xmm9,%xmm9,%xmm9
   0x00000000004007a9 <+105>:	c5 e1 ef db	vpxor  %xmm3,%xmm3,%xmm3
   0x00000000004007ad <+109>:	c5 d9 ef e4	vpxor  %xmm4,%xmm4,%xmm4
   0x00000000004007b1 <+113>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)
   0x00000000004007bb <+123>:	0f 1f 44 00 00	nopl   0x0(%rax,%rax,1)

11	    total += (data[i] == check);
   0x00000000004007c0 <+128>:	c4 e2 79 33 34 47	vpmovzxwd (%rdi,%rax,2),%xmm6
   0x00000000004007c6 <+134>:	c4 e2 79 33 7c 47 08	vpmovzxwd 0x8(%rdi,%rax,2),%xmm7
   0x00000000004007cd <+141>:	c4 e2 79 33 4c 47 10	vpmovzxwd 0x10(%rdi,%rax,2),%xmm1
   0x00000000004007d4 <+148>:	c4 e2 79 33 54 47 18	vpmovzxwd 0x18(%rdi,%rax,2),%xmm2
   0x00000000004007db <+155>:	c5 f9 76 f6	vpcmpeqd %xmm6,%xmm0,%xmm6
   0x00000000004007df <+159>:	c4 e2 7d 35 f6	vpmovzxdq %xmm6,%ymm6
   0x00000000004007e4 <+164>:	c5 cd db f5	vpand  %ymm5,%ymm6,%ymm6
   0x00000000004007e8 <+168>:	c5 bd d4 f6	vpaddq %ymm6,%ymm8,%ymm6
   0x00000000004007ec <+172>:	c5 f9 76 ff	vpcmpeqd %xmm7,%xmm0,%xmm7
   0x00000000004007f0 <+176>:	c4 e2 7d 35 ff	vpmovzxdq %xmm7,%ymm7
   0x00000000004007f5 <+181>:	c5 c5 db fd	vpand  %ymm5,%ymm7,%ymm7
   0x00000000004007f9 <+185>:	c5 b5 d4 ff	vpaddq %ymm7,%ymm9,%ymm7
   0x00000000004007fd <+189>:	c5 f9 76 c9	vpcmpeqd %xmm1,%xmm0,%xmm1
   0x0000000000400801 <+193>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x0000000000400806 <+198>:	c5 f5 db cd	vpand  %ymm5,%ymm1,%ymm1
   0x000000000040080a <+202>:	c5 65 d4 d1	vpaddq %ymm1,%ymm3,%ymm10
   0x000000000040080e <+206>:	c5 f9 76 d2	vpcmpeqd %xmm2,%xmm0,%xmm2
   0x0000000000400812 <+210>:	c4 e2 7d 35 d2	vpmovzxdq %xmm2,%ymm2
   0x0000000000400817 <+215>:	c5 ed db d5	vpand  %ymm5,%ymm2,%ymm2
   0x000000000040081b <+219>:	c5 5d d4 da	vpaddq %ymm2,%ymm4,%ymm11
   0x000000000040081f <+223>:	c4 e2 79 33 5c 47 20	vpmovzxwd 0x20(%rdi,%rax,2),%xmm3
   0x0000000000400826 <+230>:	c4 e2 79 33 64 47 28	vpmovzxwd 0x28(%rdi,%rax,2),%xmm4
   0x000000000040082d <+237>:	c4 e2 79 33 4c 47 30	vpmovzxwd 0x30(%rdi,%rax,2),%xmm1
   0x0000000000400834 <+244>:	c4 e2 79 33 54 47 38	vpmovzxwd 0x38(%rdi,%rax,2),%xmm2
   0x000000000040083b <+251>:	c5 f9 76 db	vpcmpeqd %xmm3,%xmm0,%xmm3
   0x000000000040083f <+255>:	c4 e2 7d 35 db	vpmovzxdq %xmm3,%ymm3
   0x0000000000400844 <+260>:	c5 e5 db dd	vpand  %ymm5,%ymm3,%ymm3
   0x0000000000400848 <+264>:	c5 4d d4 c3	vpaddq %ymm3,%ymm6,%ymm8
   0x000000000040084c <+268>:	c5 f9 76 dc	vpcmpeqd %xmm4,%xmm0,%xmm3
   0x0000000000400850 <+272>:	c4 e2 7d 35 db	vpmovzxdq %xmm3,%ymm3
   0x0000000000400855 <+277>:	c5 e5 db dd	vpand  %ymm5,%ymm3,%ymm3
   0x0000000000400859 <+281>:	c5 45 d4 cb	vpaddq %ymm3,%ymm7,%ymm9
   0x000000000040085d <+285>:	c5 f9 76 c9	vpcmpeqd %xmm1,%xmm0,%xmm1
   0x0000000000400861 <+289>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x0000000000400866 <+294>:	c5 f5 db cd	vpand  %ymm5,%ymm1,%ymm1
   0x000000000040086a <+298>:	c5 ad d4 d9	vpaddq %ymm1,%ymm10,%ymm3
   0x000000000040086e <+302>:	c5 f9 76 ca	vpcmpeqd %xmm2,%xmm0,%xmm1
   0x0000000000400872 <+306>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x0000000000400877 <+311>:	c5 f5 db cd	vpand  %ymm5,%ymm1,%ymm1
   0x000000000040087b <+315>:	c5 a5 d4 e1	vpaddq %ymm1,%ymm11,%ymm4

10	  for (uint64_t i = 0; i < size; i++) {
   0x000000000040087f <+319>:	48 83 c0 20	add    $0x20,%rax
   0x0000000000400883 <+323>:	48 83 c1 fe	add    $0xfffffffffffffffe,%rcx
   0x0000000000400887 <+327>:	0f 85 33 ff ff ff	jne    0x4007c0 <count_pairs+128>
   0x000000000040088d <+333>:	4d 85 c0	test   %r8,%r8
   0x0000000000400890 <+336>:	74 68	je     0x4008fa <count_pairs+442>

11	    total += (data[i] == check);
   0x0000000000400892 <+338>:	c4 e2 79 33 4c 47 18	vpmovzxwd 0x18(%rdi,%rax,2),%xmm1
   0x0000000000400899 <+345>:	c5 f9 76 c9	vpcmpeqd %xmm1,%xmm0,%xmm1
   0x000000000040089d <+349>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x00000000004008a2 <+354>:	c4 e2 7d 59 15 6d 02 00 00	vpbroadcastq 0x26d(%rip),%ymm2        # 0x400b18
   0x00000000004008ab <+363>:	c5 f5 db ca	vpand  %ymm2,%ymm1,%ymm1
   0x00000000004008af <+367>:	c5 dd d4 e1	vpaddq %ymm1,%ymm4,%ymm4
   0x00000000004008b3 <+371>:	c4 e2 79 33 4c 47 10	vpmovzxwd 0x10(%rdi,%rax,2),%xmm1
   0x00000000004008ba <+378>:	c5 f9 76 c9	vpcmpeqd %xmm1,%xmm0,%xmm1
   0x00000000004008be <+382>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x00000000004008c3 <+387>:	c5 f5 db ca	vpand  %ymm2,%ymm1,%ymm1
   0x00000000004008c7 <+391>:	c5 e5 d4 d9	vpaddq %ymm1,%ymm3,%ymm3
   0x00000000004008cb <+395>:	c4 e2 79 33 4c 47 08	vpmovzxwd 0x8(%rdi,%rax,2),%xmm1
   0x00000000004008d2 <+402>:	c5 f9 76 c9	vpcmpeqd %xmm1,%xmm0,%xmm1
   0x00000000004008d6 <+406>:	c4 e2 7d 35 c9	vpmovzxdq %xmm1,%ymm1
   0x00000000004008db <+411>:	c5 f5 db ca	vpand  %ymm2,%ymm1,%ymm1
   0x00000000004008df <+415>:	c5 35 d4 c9	vpaddq %ymm1,%ymm9,%ymm9
   0x00000000004008e3 <+419>:	c4 e2 79 33 0c 47	vpmovzxwd (%rdi,%rax,2),%xmm1
   0x00000000004008e9 <+425>:	c5 f9 76 c1	vpcmpeqd %xmm1,%xmm0,%xmm0
   0x00000000004008ed <+429>:	c4 e2 7d 35 c0	vpmovzxdq %xmm0,%ymm0
   0x00000000004008f2 <+434>:	c5 fd db c2	vpand  %ymm2,%ymm0,%ymm0
   0x00000000004008f6 <+438>:	c5 3d d4 c0	vpaddq %ymm0,%ymm8,%ymm8

10	  for (uint64_t i = 0; i < size; i++) {
   0x00000000004008fa <+442>:	c5 b5 d4 c4	vpaddq %ymm4,%ymm9,%ymm0
   0x00000000004008fe <+446>:	c5 bd d4 cb	vpaddq %ymm3,%ymm8,%ymm1
   0x0000000000400902 <+450>:	c5 f5 d4 c0	vpaddq %ymm0,%ymm1,%ymm0
   0x0000000000400906 <+454>:	c4 e3 7d 39 c1 01	vextracti128 $0x1,%ymm0,%xmm1
   0x000000000040090c <+460>:	c5 f9 d4 c1	vpaddq %xmm1,%xmm0,%xmm0
   0x0000000000400910 <+464>:	c5 f9 70 c8 4e	vpshufd $0x4e,%xmm0,%xmm1
   0x0000000000400915 <+469>:	c5 f9 d4 c1	vpaddq %xmm1,%xmm0,%xmm0
   0x0000000000400919 <+473>:	c4 e1 f9 7e c0	vmovq  %xmm0,%rax
   0x000000000040091e <+478>:	49 39 f2	cmp    %rsi,%r10
   0x0000000000400921 <+481>:	74 25	je     0x400948 <count_pairs+520>
   0x0000000000400923 <+483>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)
   0x000000000040092d <+493>:	0f 1f 00	nopl   (%rax)

11	    total += (data[i] == check);
   0x0000000000400930 <+496>:	42 0f b7 0c 57	movzwl (%rdi,%r10,2),%ecx
   0x0000000000400935 <+501>:	31 d2	xor    %edx,%edx
   0x0000000000400937 <+503>:	41 39 c9	cmp    %ecx,%r9d
   0x000000000040093a <+506>:	0f 94 c2	sete   %dl
   0x000000000040093d <+509>:	48 01 d0	add    %rdx,%rax

10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400940 <+512>:	49 ff c2	inc    %r10
   0x0000000000400943 <+515>:	4c 39 d6	cmp    %r10,%rsi
   0x0000000000400946 <+518>:	75 e8	jne    0x400930 <count_pairs+496>

12	  }
13	  return total;
   0x0000000000400948 <+520>:	c5 f8 77	vzeroupper 
   0x000000000040094b <+523>:	c3	retq   
   0x000000000040094c <+524>:	c4 41 39 ef c0	vpxor  %xmm8,%xmm8,%xmm8
   0x0000000000400951 <+529>:	31 c0	xor    %eax,%eax
   0x0000000000400953 <+531>:	c4 41 31 ef c9	vpxor  %xmm9,%xmm9,%xmm9
   0x0000000000400958 <+536>:	c5 e1 ef db	vpxor  %xmm3,%xmm3,%xmm3
   0x000000000040095c <+540>:	c5 d9 ef e4	vpxor  %xmm4,%xmm4,%xmm4

10	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400960 <+544>:	4d 85 c0	test   %r8,%r8
   0x0000000000400963 <+547>:	0f 85 29 ff ff ff	jne    0x400892 <count_pairs+338>
   0x0000000000400969 <+553>:	eb 8f	jmp    0x4008fa <count_pairs+442>
End of assembler dump.

```

It becomes a giant mess

We need multiple versions of the loops depending on the number of iterations

Sometimes we need to do special work at the beginning or end if the alignment is not proper

We can fix some of these issues by giving the compiler more information.
Here is the same code, but with some extra information given to the compiler

```c++
uint64_t count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
  // tell the compilier that data is aligned to 32 bytes
  data = __builtin_assume_aligned(data, 32);
  // tell the compilier that size is a multiple of 32
  size = size & (~31U);
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size; i++) {
    total += (data[i] == check);
  }
  return total;
}
```

In this case the start and end work is trivial, so it runs at the same speed, but if we look at the assembly, it is a bit simpler 

```
Dump of assembler code for function count_pairs:
ex1b.c:
7	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
   0x0000000000400740 <+0>:	b8 e0 ff ff ff	mov    $0xffffffe0,%eax

8	  // tell the compilier that data is aligned to 32 bytes
9	  data = __builtin_assume_aligned(data, 32);
10	  // tell the compilier that size is a multiple of 32
11	  size = size & (~31U);
12	  uint64_t total = 0;
13	  uint16_t check = target | (target << 8U);
14	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400745 <+5>:	48 21 f0	and    %rsi,%rax
   0x0000000000400748 <+8>:	0f 84 2f 01 00 00	je     0x40087d <count_pairs+317>
   0x000000000040074e <+14>:	89 d1	mov    %edx,%ecx
   0x0000000000400750 <+16>:	c1 e1 08	shl    $0x8,%ecx
   0x0000000000400753 <+19>:	09 d1	or     %edx,%ecx
   0x0000000000400755 <+21>:	c5 f9 6e c1	vmovd  %ecx,%xmm0
   0x0000000000400759 <+25>:	c4 e2 79 58 c0	vpbroadcastd %xmm0,%xmm0
   0x000000000040075e <+30>:	48 83 c0 f0	add    $0xfffffffffffffff0,%rax
   0x0000000000400762 <+34>:	48 c1 e8 04	shr    $0x4,%rax
   0x0000000000400766 <+38>:	48 ff c0	inc    %rax
   0x0000000000400769 <+41>:	48 83 c7 38	add    $0x38,%rdi
   0x000000000040076d <+45>:	c4 41 39 ef c0	vpxor  %xmm8,%xmm8,%xmm8
   0x0000000000400772 <+50>:	c4 e2 7d 59 0d ad 02 00 00	vpbroadcastq 0x2ad(%rip),%ymm1        # 0x400a28
   0x000000000040077b <+59>:	c5 e1 ef db	vpxor  %xmm3,%xmm3,%xmm3
   0x000000000040077f <+63>:	c5 d9 ef e4	vpxor  %xmm4,%xmm4,%xmm4
   0x0000000000400783 <+67>:	c5 d1 ef ed	vpxor  %xmm5,%xmm5,%xmm5
   0x0000000000400787 <+71>:	66 0f 1f 84 00 00 00 00 00	nopw   0x0(%rax,%rax,1)

15	    total += (data[i] == check);
   0x0000000000400790 <+80>:	c4 e2 79 33 77 c8	vpmovzxwd -0x38(%rdi),%xmm6
   0x0000000000400796 <+86>:	c4 e2 79 33 7f d0	vpmovzxwd -0x30(%rdi),%xmm7
   0x000000000040079c <+92>:	c4 e2 79 33 57 d8	vpmovzxwd -0x28(%rdi),%xmm2
   0x00000000004007a2 <+98>:	c4 62 79 33 4f e0	vpmovzxwd -0x20(%rdi),%xmm9
   0x00000000004007a8 <+104>:	c5 f9 76 f6	vpcmpeqd %xmm6,%xmm0,%xmm6
   0x00000000004007ac <+108>:	c4 e2 7d 35 f6	vpmovzxdq %xmm6,%ymm6
   0x00000000004007b1 <+113>:	c5 cd db f1	vpand  %ymm1,%ymm6,%ymm6
   0x00000000004007b5 <+117>:	c5 bd d4 f6	vpaddq %ymm6,%ymm8,%ymm6
   0x00000000004007b9 <+121>:	c5 f9 76 ff	vpcmpeqd %xmm7,%xmm0,%xmm7
   0x00000000004007bd <+125>:	c4 e2 7d 35 ff	vpmovzxdq %xmm7,%ymm7
   0x00000000004007c2 <+130>:	c5 c5 db f9	vpand  %ymm1,%ymm7,%ymm7
   0x00000000004007c6 <+134>:	c5 e5 d4 df	vpaddq %ymm7,%ymm3,%ymm3
   0x00000000004007ca <+138>:	c5 f9 76 d2	vpcmpeqd %xmm2,%xmm0,%xmm2
   0x00000000004007ce <+142>:	c4 e2 7d 35 d2	vpmovzxdq %xmm2,%ymm2
   0x00000000004007d3 <+147>:	c5 ed db d1	vpand  %ymm1,%ymm2,%ymm2
   0x00000000004007d7 <+151>:	c5 5d d4 d2	vpaddq %ymm2,%ymm4,%ymm10
   0x00000000004007db <+155>:	c5 b1 76 e0	vpcmpeqd %xmm0,%xmm9,%xmm4
   0x00000000004007df <+159>:	c4 e2 7d 35 e4	vpmovzxdq %xmm4,%ymm4
   0x00000000004007e4 <+164>:	c5 dd db e1	vpand  %ymm1,%ymm4,%ymm4
   0x00000000004007e8 <+168>:	c5 55 d4 cc	vpaddq %ymm4,%ymm5,%ymm9
   0x00000000004007ec <+172>:	c4 e2 79 33 67 e8	vpmovzxwd -0x18(%rdi),%xmm4
   0x00000000004007f2 <+178>:	c4 e2 79 33 7f f0	vpmovzxwd -0x10(%rdi),%xmm7
   0x00000000004007f8 <+184>:	c4 e2 79 33 57 f8	vpmovzxwd -0x8(%rdi),%xmm2
   0x00000000004007fe <+190>:	c4 e2 79 33 2f	vpmovzxwd (%rdi),%xmm5
   0x0000000000400803 <+195>:	c5 f9 76 e4	vpcmpeqd %xmm4,%xmm0,%xmm4
   0x0000000000400807 <+199>:	c4 e2 7d 35 e4	vpmovzxdq %xmm4,%ymm4
   0x000000000040080c <+204>:	c5 dd db e1	vpand  %ymm1,%ymm4,%ymm4
   0x0000000000400810 <+208>:	c5 4d d4 c4	vpaddq %ymm4,%ymm6,%ymm8
   0x0000000000400814 <+212>:	c5 f9 76 e7	vpcmpeqd %xmm7,%xmm0,%xmm4
   0x0000000000400818 <+216>:	c4 e2 7d 35 e4	vpmovzxdq %xmm4,%ymm4
   0x000000000040081d <+221>:	c5 dd db e1	vpand  %ymm1,%ymm4,%ymm4
   0x0000000000400821 <+225>:	c5 e5 d4 dc	vpaddq %ymm4,%ymm3,%ymm3
   0x0000000000400825 <+229>:	c5 f9 76 d2	vpcmpeqd %xmm2,%xmm0,%xmm2
   0x0000000000400829 <+233>:	c4 e2 7d 35 d2	vpmovzxdq %xmm2,%ymm2
   0x000000000040082e <+238>:	c5 ed db d1	vpand  %ymm1,%ymm2,%ymm2
   0x0000000000400832 <+242>:	c5 ad d4 e2	vpaddq %ymm2,%ymm10,%ymm4
   0x0000000000400836 <+246>:	c5 f9 76 d5	vpcmpeqd %xmm5,%xmm0,%xmm2
   0x000000000040083a <+250>:	c4 e2 7d 35 d2	vpmovzxdq %xmm2,%ymm2
   0x000000000040083f <+255>:	c5 ed db d1	vpand  %ymm1,%ymm2,%ymm2
   0x0000000000400843 <+259>:	c5 b5 d4 ea	vpaddq %ymm2,%ymm9,%ymm5

14	  for (uint64_t i = 0; i < size; i++) {
   0x0000000000400847 <+263>:	48 83 c7 40	add    $0x40,%rdi
   0x000000000040084b <+267>:	48 83 c0 fe	add    $0xfffffffffffffffe,%rax
   0x000000000040084f <+271>:	0f 85 3b ff ff ff	jne    0x400790 <count_pairs+80>
   0x0000000000400855 <+277>:	c5 bd d4 c3	vpaddq %ymm3,%ymm8,%ymm0
   0x0000000000400859 <+281>:	c5 dd d4 c0	vpaddq %ymm0,%ymm4,%ymm0
   0x000000000040085d <+285>:	c5 d5 d4 c0	vpaddq %ymm0,%ymm5,%ymm0
   0x0000000000400861 <+289>:	c4 e3 7d 39 c1 01	vextracti128 $0x1,%ymm0,%xmm1
   0x0000000000400867 <+295>:	c5 f9 d4 c1	vpaddq %xmm1,%xmm0,%xmm0
   0x000000000040086b <+299>:	c5 f9 70 c8 4e	vpshufd $0x4e,%xmm0,%xmm1
   0x0000000000400870 <+304>:	c5 f9 d4 c1	vpaddq %xmm1,%xmm0,%xmm0
   0x0000000000400874 <+308>:	c4 e1 f9 7e c0	vmovq  %xmm0,%rax

16	  }
17	  return total;
   0x0000000000400879 <+313>:	c5 f8 77	vzeroupper 
   0x000000000040087c <+316>:	c3	retq   
   0x000000000040087d <+317>:	31 c0	xor    %eax,%eax
   0x000000000040087f <+319>:	c3	retq   
End of assembler dump.

```

The compiler can do a lot to optimize and use the vector registers to speed up the code, but we can also do it manually.

Intrinsics are basically functions which will compile down to a single special assembly instruction that we can use to use the vector instrctions in c or c++ code without having to write raw assembly.  The list of them can be found [https://software.intel.com/sites/landingpage/IntrinsicsGuide/](https://software.intel.com/sites/landingpage/IntrinsicsGuide/)

![_mm256_cmpeq_epi16](Lec09_source/_mm256_cmpeq_epi16.JPG "_mm256_cmpeq_epi16")




Here we manually vectorize this code 
```c++
uint64_t count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
  data = __builtin_assume_aligned(data, 32);
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  // set up a vector so that you can comare the pair of bytes in all 16 positions
  __m256i compare = _mm256_set1_epi16(check);
  for (uint64_t i = 0; i < size; i += 16) {
    // do 16 comparisons at once
    uint32_t block = _mm256_movemask_epi8( _mm256_cmpeq_epi16(_mm256_load_si256((__m256i *)(data + i)), compare));
    // once we move the result of the comparison into a normal register, count the set bits
    total += __builtin_popcount(block);
  }
  return total / 2;
}
```

- _mm256_set1_epi16 takes in a 16-bit object and puts it at all 16 16-bit locations in a 256 bit register 
- _mm256_load_si256 loads 256 bits of data from the given memory address
- _mm256_cmpeq_epi16 compares the two vectors element wise
- _mm256_movemask_epi8 takes the top bit from each byte into a 32-bit object, we use it to get our data out of the vector register into the normal registers
- __builtin_popcount counts the number of set bits in a normal 32-bit register


Most of these require the `avx2` flag above, but popcount requires the special `popcnt` flag

So what we are doing here, is for each set of 16 elements, we compare if they are equal to the vector of a pair of target bytes, then we move these out into a normal register, we then count up the number of equal shorts with the pop count, however, since we moved it out by bytes and not shorts we need to divide by 2 at the end.

We get fairly good performance with a significant improvement over the compiler

| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 994 |
| O3 | 208 |
| march=native | 97 |
| manual vectorization | 56 |

Let us take a look at the assembly generated this time.

```
Dump of assembler code for function count_pairs:
ex1c.c:
8	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
9	  data = __builtin_assume_aligned(data, 32);
10	  uint64_t total = 0;
11	  uint16_t check = target | (target << 8U);
12	  __m256i compare = _mm256_set1_epi16(check);
13	  for (uint64_t i = 0; i < size; i += 16) {
   0x0000000000400740 <+0>:	48 85 f6	test   %rsi,%rsi
   0x0000000000400743 <+3>:	74 3b	je     0x400780 <count_pairs+64>
   0x0000000000400745 <+5>:	89 d0	mov    %edx,%eax
   0x0000000000400747 <+7>:	c1 e0 08	shl    $0x8,%eax
   0x000000000040074a <+10>:	09 d0	or     %edx,%eax
   0x000000000040074c <+12>:	c5 f9 6e c0	vmovd  %eax,%xmm0
   0x0000000000400750 <+16>:	c4 e2 7d 79 c0	vpbroadcastw %xmm0,%ymm0
   0x0000000000400755 <+21>:	31 c9	xor    %ecx,%ecx
   0x0000000000400757 <+23>:	31 c0	xor    %eax,%eax
   0x0000000000400759 <+25>:	0f 1f 80 00 00 00 00	nopl   0x0(%rax)

15	        _mm256_cmpeq_epi16(_mm256_load_si256((__m256i *)(data + i)), compare));
   0x0000000000400760 <+32>:	c5 fd 75 0c 4f	vpcmpeqw (%rdi,%rcx,2),%ymm0,%ymm1

14	    uint32_t block = _mm256_movemask_epi8(
   0x0000000000400765 <+37>:	c5 fd d7 d1	vpmovmskb %ymm1,%edx

16	    total += __builtin_popcount(block);
   0x0000000000400769 <+41>:	f3 0f b8 d2	popcnt %edx,%edx
   0x000000000040076d <+45>:	48 01 d0	add    %rdx,%rax

13	  for (uint64_t i = 0; i < size; i += 16) {
   0x0000000000400770 <+48>:	48 83 c1 10	add    $0x10,%rcx
   0x0000000000400774 <+52>:	48 39 f1	cmp    %rsi,%rcx
   0x0000000000400777 <+55>:	72 e7	jb     0x400760 <count_pairs+32>

17	  }
18	  return total / 2;
   0x0000000000400779 <+57>:	48 d1 e8	shr    %rax
   0x000000000040077c <+60>:	c5 f8 77	vzeroupper 
   0x000000000040077f <+63>:	c3	retq   
   0x0000000000400780 <+64>:	31 c0	xor    %eax,%eax
   0x0000000000400782 <+66>:	c3	retq   
End of assembler dump.

```

This ends up being one of the simpler assembly versions we have seen so far

This is mostly due to how it generated basically just what we told it to and nothing else.

One thing we note is that this version doesn't have any form of loop optimizations such as unrolling, once we start adding manual vectorization this gets harder for the compiler.

I manually do some loop unrolling in the following version 

```c++
uint64_t __attribute__((noinline))
count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
  data = __builtin_assume_aligned(data, 32);
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  __m256i compare = _mm256_set1_epi16(check);
  for (uint64_t j = 0; j < size; j += 1024) {
    __m256i eq_vec = _mm256_setzero_si256();
    for (uint64_t i = 0; i < 1024; i += 64) {
      __m256i a = _mm256_cmpeq_epi16( _mm256_load_si256((__m256i *)(data + j + i)), compare);
      __m256i b = _mm256_cmpeq_epi16( _mm256_load_si256((__m256i *)(data + j + i)), compare);
      __m256i c = _mm256_cmpeq_epi16( _mm256_load_si256((__m256i *)(data + j + i)), compare);
      __m256i d = _mm256_cmpeq_epi16( _mm256_load_si256((__m256i *)(data + j + i)), compare);
      __m256i e = _mm256_add_epi16(a, b);
      __m256i f = _mm256_add_epi16(c, d);
      __m256i g = _mm256_add_epi16(e, f);
      // notice that in the inner loop I do not combine the result from the different lanes
      eq_vec = _mm256_sub_epi16(eq_vec, g);
    }
    // only combine the lanes rarely
    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
    total += _mm256_extract_epi16(eq_vec, 0);
    total += _mm256_extract_epi16(eq_vec, 8);
  }
  return total;
}
```

Honestly this version becomes a mess, but it is fast.

| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 994 |
| O3 | 208 |
| march=native | 97 |
| manual vectorization | 56 |
| manual vectorization with manual loop unrolling | 48|

One thing to pay attention to is that it is costly to move data between vector and normal registers so we try and limit how often that needs to be done.

Another thing to notice is that this code went from easy and readable, and also maintainably to something nobody ever wants to touch

The assembly is below for your reference.




```
Dump of assembler code for function count_pairs:
ex1d.c:
8	count_pairs(uint16_t *data, uint64_t size, uint8_t target) {
9	  data = __builtin_assume_aligned(data, 32);
10	  uint64_t total = 0;
11	  uint16_t check = target | (target << 8U);
12	  __m256i compare = _mm256_set1_epi16(check);
13	  for (uint64_t j = 0; j < size; j += 1024) {
   0x0000000000400740 <+0>:	48 85 f6	test   %rsi,%rsi
   0x0000000000400743 <+3>:	0f 84 7c 01 00 00	je     0x4008c5 <count_pairs+389>
   0x0000000000400749 <+9>:	89 d0	mov    %edx,%eax
   0x000000000040074b <+11>:	c1 e0 08	shl    $0x8,%eax
   0x000000000040074e <+14>:	09 d0	or     %edx,%eax
   0x0000000000400750 <+16>:	c5 f9 6e c0	vmovd  %eax,%xmm0
   0x0000000000400754 <+20>:	c4 e2 7d 79 c0	vpbroadcastw %xmm0,%ymm0
   0x0000000000400759 <+25>:	31 c0	xor    %eax,%eax
   0x000000000040075b <+27>:	c5 f1 ef c9	vpxor  %xmm1,%xmm1,%xmm1
   0x000000000040075f <+31>:	31 c9	xor    %ecx,%ecx
   0x0000000000400761 <+33>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)
   0x000000000040076b <+43>:	0f 1f 44 00 00	nopl   0x0(%rax,%rax,1)

14	    __m256i eq_vec = _mm256_setzero_si256();
15	    for (uint64_t i = 0; i < 1024; i += 64) {
16	      __m256i a = _mm256_cmpeq_epi16(
   0x0000000000400770 <+48>:	c5 fd 75 14 4f	vpcmpeqw (%rdi,%rcx,2),%ymm0,%ymm2
   0x0000000000400775 <+53>:	c5 fd 75 9c 4f 80 00 00 00	vpcmpeqw 0x80(%rdi,%rcx,2),%ymm0,%ymm3

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x000000000040077e <+62>:	c5 ed 71 f2 02	vpsllw $0x2,%ymm2,%ymm2
   0x0000000000400783 <+67>:	c5 e5 71 f3 02	vpsllw $0x2,%ymm3,%ymm3

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x0000000000400788 <+72>:	c5 ed fd d3	vpaddw %ymm3,%ymm2,%ymm2

16	      __m256i a = _mm256_cmpeq_epi16(
   0x000000000040078c <+76>:	c5 fd 75 9c 4f 00 01 00 00	vpcmpeqw 0x100(%rdi,%rcx,2),%ymm0,%ymm3

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400795 <+85>:	c5 e5 71 f3 02	vpsllw $0x2,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x000000000040079a <+90>:	c5 fd 75 a4 4f 80 01 00 00	vpcmpeqw 0x180(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007a3 <+99>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x00000000004007a8 <+104>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x00000000004007ac <+108>:	c5 fd 75 a4 4f 00 02 00 00	vpcmpeqw 0x200(%rdi,%rcx,2),%ymm0,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x00000000004007b5 <+117>:	c5 ed fd d3	vpaddw %ymm3,%ymm2,%ymm2

26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007b9 <+121>:	c5 e5 71 f4 02	vpsllw $0x2,%ymm4,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x00000000004007be <+126>:	c5 fd 75 a4 4f 80 02 00 00	vpcmpeqw 0x280(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007c7 <+135>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x00000000004007cc <+140>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x00000000004007d0 <+144>:	c5 fd 75 a4 4f 00 03 00 00	vpcmpeqw 0x300(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007d9 <+153>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x00000000004007de <+158>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x00000000004007e2 <+162>:	c5 fd 75 a4 4f 80 03 00 00	vpcmpeqw 0x380(%rdi,%rcx,2),%ymm0,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x00000000004007eb <+171>:	c5 ed fd d3	vpaddw %ymm3,%ymm2,%ymm2

26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007ef <+175>:	c5 e5 71 f4 02	vpsllw $0x2,%ymm4,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x00000000004007f4 <+180>:	c5 fd 75 a4 4f 00 04 00 00	vpcmpeqw 0x400(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x00000000004007fd <+189>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x0000000000400802 <+194>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x0000000000400806 <+198>:	c5 fd 75 a4 4f 80 04 00 00	vpcmpeqw 0x480(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x000000000040080f <+207>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x0000000000400814 <+212>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x0000000000400818 <+216>:	c5 fd 75 a4 4f 00 05 00 00	vpcmpeqw 0x500(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400821 <+225>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x0000000000400826 <+230>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3
   0x000000000040082a <+234>:	c5 ed fd d3	vpaddw %ymm3,%ymm2,%ymm2

16	      __m256i a = _mm256_cmpeq_epi16(
   0x000000000040082e <+238>:	c5 fd 75 9c 4f 80 05 00 00	vpcmpeqw 0x580(%rdi,%rcx,2),%ymm0,%ymm3

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400837 <+247>:	c5 e5 71 f3 02	vpsllw $0x2,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x000000000040083c <+252>:	c5 fd 75 a4 4f 00 06 00 00	vpcmpeqw 0x600(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400845 <+261>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x000000000040084a <+266>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x000000000040084e <+270>:	c5 fd 75 a4 4f 80 06 00 00	vpcmpeqw 0x680(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400857 <+279>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x000000000040085c <+284>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x0000000000400860 <+288>:	c5 fd 75 a4 4f 00 07 00 00	vpcmpeqw 0x700(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x0000000000400869 <+297>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x000000000040086e <+302>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3

16	      __m256i a = _mm256_cmpeq_epi16(
   0x0000000000400872 <+306>:	c5 fd 75 a4 4f 80 07 00 00	vpcmpeqw 0x780(%rdi,%rcx,2),%ymm0,%ymm4

17	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
18	      __m256i b = _mm256_cmpeq_epi16(
19	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
20	      __m256i c = _mm256_cmpeq_epi16(
21	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
22	      __m256i d = _mm256_cmpeq_epi16(
23	          _mm256_load_si256((__m256i *)(data + j + i)), compare);
24	      __m256i e = _mm256_add_epi16(a, b);
25	      __m256i f = _mm256_add_epi16(c, d);
26	      __m256i g = _mm256_add_epi16(e, f);
   0x000000000040087b <+315>:	c5 dd 71 f4 02	vpsllw $0x2,%ymm4,%ymm4

27	      eq_vec = _mm256_sub_epi16(eq_vec, g);
   0x0000000000400880 <+320>:	c5 e5 fd dc	vpaddw %ymm4,%ymm3,%ymm3
   0x0000000000400884 <+324>:	c5 ed fd d3	vpaddw %ymm3,%ymm2,%ymm2
   0x0000000000400888 <+328>:	c5 f5 f9 d2	vpsubw %ymm2,%ymm1,%ymm2

28	    }
29	    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
   0x000000000040088c <+332>:	c4 e2 6d 01 d2	vphaddw %ymm2,%ymm2,%ymm2

30	    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
   0x0000000000400891 <+337>:	c4 e2 6d 01 d2	vphaddw %ymm2,%ymm2,%ymm2

31	    eq_vec = _mm256_hadd_epi16(eq_vec, eq_vec);
   0x0000000000400896 <+342>:	c4 e2 6d 01 d2	vphaddw %ymm2,%ymm2,%ymm2

32	    total += _mm256_extract_epi16(eq_vec, 0);
   0x000000000040089b <+347>:	c5 f9 c5 d2 00	vpextrw $0x0,%xmm2,%edx
   0x00000000004008a0 <+352>:	48 01 c2	add    %rax,%rdx

33	    total += _mm256_extract_epi16(eq_vec, 8);
   0x00000000004008a3 <+355>:	c4 e3 7d 39 d2 01	vextracti128 $0x1,%ymm2,%xmm2
   0x00000000004008a9 <+361>:	c5 f9 c5 c2 00	vpextrw $0x0,%xmm2,%eax
   0x00000000004008ae <+366>:	48 01 d0	add    %rdx,%rax

13	  for (uint64_t j = 0; j < size; j += 1024) {
   0x00000000004008b1 <+369>:	48 81 c1 00 04 00 00	add    $0x400,%rcx
   0x00000000004008b8 <+376>:	48 39 f1	cmp    %rsi,%rcx
   0x00000000004008bb <+379>:	0f 82 af fe ff ff	jb     0x400770 <count_pairs+48>

34	  }
35	  return total;
   0x00000000004008c1 <+385>:	c5 f8 77	vzeroupper 
   0x00000000004008c4 <+388>:	c3	retq   
   0x00000000004008c5 <+389>:	31 c0	xor    %eax,%eax
   0x00000000004008c7 <+391>:	c3	retq   
End of assembler dump.

```

| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 994 |
| O3 | 208 |
| march=native | 97 |
| manual vectorization | 56 |
| manual vectorization with manual loop unrolling | 48|

So all together we are able to get a 20x speedup, 4x over optimized code, and 2x over the best the compiler could do, but it takes a lot more effort to write the last two versions than the first three.

# Problem 2

Now we will look at a slight variation of the problem.

We remove the requirement that the pairs are aligned, before the pairs had to be in even - odd postions, not they can be either even-odd, or odd-even.

The code for this is not much more complicated, we look at every byte position now instead of each short position.

```c++
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  uint16_t check = target | (target << 8U);
  for (uint64_t i = 0; i < size * 2 - 1; i++) {
    total += (load16(data + i) == check);
  }
  return total;
}
```


| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 2897 |
| O3 | 610 |
| march=native | 584|

Unoptimized 
```
Dump of assembler code for function count_pairs:
ex2a.c:
7	count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
   0x0000000000400780 <+0>:	55	push   %rbp
   0x0000000000400781 <+1>:	48 89 e5	mov    %rsp,%rbp
   0x0000000000400784 <+4>:	48 83 ec 30	sub    $0x30,%rsp
   0x0000000000400788 <+8>:	48 89 7d f8	mov    %rdi,-0x8(%rbp)
   0x000000000040078c <+12>:	48 89 75 f0	mov    %rsi,-0x10(%rbp)
   0x0000000000400790 <+16>:	88 55 ef	mov    %dl,-0x11(%rbp)

8	  uint64_t total = 0;
   0x0000000000400793 <+19>:	48 c7 45 e0 00 00 00 00	movq   $0x0,-0x20(%rbp)

9	  uint16_t check = target | (target << 8U);
   0x000000000040079b <+27>:	0f b6 45 ef	movzbl -0x11(%rbp),%eax
   0x000000000040079f <+31>:	0f b6 4d ef	movzbl -0x11(%rbp),%ecx
   0x00000000004007a3 <+35>:	c1 e1 08	shl    $0x8,%ecx
   0x00000000004007a6 <+38>:	09 c8	or     %ecx,%eax
   0x00000000004007a8 <+40>:	66 89 45 de	mov    %ax,-0x22(%rbp)

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x00000000004007ac <+44>:	48 c7 45 d0 00 00 00 00	movq   $0x0,-0x30(%rbp)
   0x00000000004007b4 <+52>:	48 8b 45 d0	mov    -0x30(%rbp),%rax
   0x00000000004007b8 <+56>:	48 8b 4d f0	mov    -0x10(%rbp),%rcx
   0x00000000004007bc <+60>:	48 c1 e1 01	shl    $0x1,%rcx
   0x00000000004007c0 <+64>:	48 81 e9 01 00 00 00	sub    $0x1,%rcx
   0x00000000004007c7 <+71>:	48 39 c8	cmp    %rcx,%rax
   0x00000000004007ca <+74>:	0f 83 43 00 00 00	jae    0x400813 <count_pairs+147>

11	    total += (load16(data + i) == check);
   0x00000000004007d0 <+80>:	48 8b 45 f8	mov    -0x8(%rbp),%rax
   0x00000000004007d4 <+84>:	48 03 45 d0	add    -0x30(%rbp),%rax
   0x00000000004007d8 <+88>:	48 89 c7	mov    %rax,%rdi
   0x00000000004007db <+91>:	e8 80 ff ff ff	callq  0x400760 <load16>
   0x00000000004007e0 <+96>:	0f b7 c8	movzwl %ax,%ecx
   0x00000000004007e3 <+99>:	0f b7 55 de	movzwl -0x22(%rbp),%edx
   0x00000000004007e7 <+103>:	39 d1	cmp    %edx,%ecx
   0x00000000004007e9 <+105>:	40 0f 94 c6	sete   %sil
   0x00000000004007ed <+109>:	40 80 e6 01	and    $0x1,%sil
   0x00000000004007f1 <+113>:	40 0f b6 ce	movzbl %sil,%ecx
   0x00000000004007f5 <+117>:	48 63 f9	movslq %ecx,%rdi
   0x00000000004007f8 <+120>:	48 03 7d e0	add    -0x20(%rbp),%rdi
   0x00000000004007fc <+124>:	48 89 7d e0	mov    %rdi,-0x20(%rbp)

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x0000000000400800 <+128>:	48 8b 45 d0	mov    -0x30(%rbp),%rax
   0x0000000000400804 <+132>:	48 05 01 00 00 00	add    $0x1,%rax
   0x000000000040080a <+138>:	48 89 45 d0	mov    %rax,-0x30(%rbp)
   0x000000000040080e <+142>:	e9 a1 ff ff ff	jmpq   0x4007b4 <count_pairs+52>

12	  }
13	  return total;
   0x0000000000400813 <+147>:	48 8b 45 e0	mov    -0x20(%rbp),%rax
   0x0000000000400817 <+151>:	48 83 c4 30	add    $0x30,%rsp
   0x000000000040081b <+155>:	5d	pop    %rbp
   0x000000000040081c <+156>:	c3	retq   
End of assembler dump.
```

O3
```
Dump of assembler code for function count_pairs:
ex2a.c:
7	count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
8	  uint64_t total = 0;
9	  uint16_t check = target | (target << 8U);
   0x0000000000400740 <+0>:	41 89 d1	mov    %edx,%r9d
   0x0000000000400743 <+3>:	41 c1 e1 08	shl    $0x8,%r9d
   0x0000000000400747 <+7>:	41 09 d1	or     %edx,%r9d

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x000000000040074a <+10>:	4c 8d 04 36	lea    (%rsi,%rsi,1),%r8
   0x000000000040074e <+14>:	49 83 c0 ff	add    $0xffffffffffffffff,%r8
   0x0000000000400752 <+18>:	31 f6	xor    %esi,%esi
   0x0000000000400754 <+20>:	31 c0	xor    %eax,%eax
   0x0000000000400756 <+22>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)

./helpers.h:
30	  memcpy(&data, loc, sizeof(data));
   0x0000000000400760 <+32>:	0f b7 14 37	movzwl (%rdi,%rsi,1),%edx

ex2a.c:
11	    total += (load16(data + i) == check);
   0x0000000000400764 <+36>:	31 c9	xor    %ecx,%ecx
   0x0000000000400766 <+38>:	41 39 d1	cmp    %edx,%r9d
   0x0000000000400769 <+41>:	0f 94 c1	sete   %cl
   0x000000000040076c <+44>:	48 01 c8	add    %rcx,%rax

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x000000000040076f <+47>:	48 83 c6 01	add    $0x1,%rsi
   0x0000000000400773 <+51>:	4c 39 c6	cmp    %r8,%rsi
   0x0000000000400776 <+54>:	72 e8	jb     0x400760 <count_pairs+32>

12	  }
13	  return total;
   0x0000000000400778 <+56>:	c3	retq   
End of assembler dump.
```

We see how it is able to simplify the inner loop as before, but it no longer unrolls the loop at all

march=native
```
Dump of assembler code for function count_pairs:
ex2a.c:
7	count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
8	  uint64_t total = 0;
9	  uint16_t check = target | (target << 8U);
   0x0000000000400740 <+0>:	41 89 d1	mov    %edx,%r9d
   0x0000000000400743 <+3>:	41 c1 e1 08	shl    $0x8,%r9d
   0x0000000000400747 <+7>:	41 09 d1	or     %edx,%r9d

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x000000000040074a <+10>:	4c 8d 04 36	lea    (%rsi,%rsi,1),%r8
   0x000000000040074e <+14>:	49 ff c8	dec    %r8
   0x0000000000400751 <+17>:	31 f6	xor    %esi,%esi
   0x0000000000400753 <+19>:	31 c0	xor    %eax,%eax
   0x0000000000400755 <+21>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)
   0x000000000040075f <+31>:	90	nop

./helpers.h:
30	  memcpy(&data, loc, sizeof(data));
   0x0000000000400760 <+32>:	0f b7 14 37	movzwl (%rdi,%rsi,1),%edx

ex2a.c:
11	    total += (load16(data + i) == check);
   0x0000000000400764 <+36>:	31 c9	xor    %ecx,%ecx
   0x0000000000400766 <+38>:	41 39 d1	cmp    %edx,%r9d
   0x0000000000400769 <+41>:	0f 94 c1	sete   %cl
   0x000000000040076c <+44>:	48 01 c8	add    %rcx,%rax

10	  for (uint64_t i = 0; i < size * 2 - 1; i++) {
   0x000000000040076f <+47>:	48 ff c6	inc    %rsi
   0x0000000000400772 <+50>:	4c 39 c6	cmp    %r8,%rsi
   0x0000000000400775 <+53>:	72 e9	jb     0x400760 <count_pairs+32>

12	  }
13	  return total;
   0x0000000000400777 <+55>:	c3	retq   
End of assembler dump.

```

The only difference here is the use of inc and dec instead of just addition and subtraction which results in smaller instructions and doesn't mess with as many flags

| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 2897 |
| O3 | 610 |
| march=native | 584|

Here the work is about twice as much, so the first two numbers seem reasonable, but now the compiler is unable to gain benefit from vectorization.

This is because without the alignment restriction this is a much messier problem.



However, we can still do it by hand.

```c++
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  uint32_t last_bit = 0;
  __m256i compare = _mm256_set1_epi8(target);
  for (uint64_t i = 0; i < size * 2; i += 32) {
    uint32_t block = _mm256_movemask_epi8( _mm256_cmpeq_epi8(_mm256_load_si256((__m256i *)(data + i)), compare));
    total += __builtin_popcount(block & (block >> 1U));
    if (last_bit) {
      total += last_bit & block;
    }
    last_bit = block >> 31U;
  }
  return total;
}
```

Let us walk through the loop together

We first do 
```c++
uint32_t block = _mm256_movemask_epi8(_mm256_cmpeq_epi8(_mm256_load_si256((__m256i *)(data + i)), compare));
```
Which tells us which byte is equal to the target byte and moves this data into a normal 32-bit register

We then do 
`block & (block >> 1U)`
Which actually uses bit level parallelism and results in a 1 exactly where both that location, and the location to the left are equal to the target byte, exactly what we are looking for.

Then we use pop count to see how many of these pairs there was.

Lastly we need to do some extra bookkeeping to manage when pairs go between vectors of data.


| Version | time to process 1 Gib in ms |
|--- | --- |
| unoptimized | 2897 |
| O3 | 610 |
| march=native | 584|
| manual | 69|

Dump of assembler code for function count_pairs:
```
ex2b.c:
8	count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
   0x0000000000400740 <+0>:	31 c9	xor    %ecx,%ecx

9	  uint64_t total = 0;
10	  uint32_t last_bit = 0;
11	  __m256i compare = _mm256_set1_epi8(target);
12	  for (uint64_t i = 0; i < size * 2; i += 32) {
   0x0000000000400742 <+2>:	48 01 f6	add    %rsi,%rsi
   0x0000000000400745 <+5>:	74 4d	je     0x400794 <count_pairs+84>
   0x0000000000400747 <+7>:	c5 f9 6e c2	vmovd  %edx,%xmm0
   0x000000000040074b <+11>:	c4 e2 7d 78 c0	vpbroadcastb %xmm0,%ymm0
   0x0000000000400750 <+16>:	45 31 c0	xor    %r8d,%r8d
   0x0000000000400753 <+19>:	45 31 c9	xor    %r9d,%r9d
   0x0000000000400756 <+22>:	66 2e 0f 1f 84 00 00 00 00 00	nopw   %cs:0x0(%rax,%rax,1)

13	    uint32_t block = _mm256_movemask_epi8(
   0x0000000000400760 <+32>:	c4 a1 7d 74 0c 0f	vpcmpeqb (%rdi,%r9,1),%ymm0,%ymm1
   0x0000000000400766 <+38>:	c5 fd d7 d1	vpmovmskb %ymm1,%edx

14	        _mm256_cmpeq_epi8(_mm256_load_si256((__m256i *)(data + i)), compare));
15	    total += __builtin_popcount(block & (block >> 1U));
   0x000000000040076a <+42>:	89 d0	mov    %edx,%eax
   0x000000000040076c <+44>:	d1 e8	shr    %eax
   0x000000000040076e <+46>:	21 d0	and    %edx,%eax
   0x0000000000400770 <+48>:	f3 0f b8 c0	popcnt %eax,%eax

16	    if (last_bit) {
   0x0000000000400774 <+52>:	21 d1	and    %edx,%ecx

15	    total += __builtin_popcount(block & (block >> 1U));
   0x0000000000400776 <+54>:	4c 01 c1	add    %r8,%rcx

16	    if (last_bit) {
   0x0000000000400779 <+57>:	48 01 c1	add    %rax,%rcx

17	      total += last_bit & block;
18	    }
19	    last_bit = block >> 31U;
   0x000000000040077c <+60>:	c1 ea 1f	shr    $0x1f,%edx

12	  for (uint64_t i = 0; i < size * 2; i += 32) {
   0x000000000040077f <+63>:	49 83 c1 20	add    $0x20,%r9
   0x0000000000400783 <+67>:	48 89 c8	mov    %rcx,%rax
   0x0000000000400786 <+70>:	49 89 c8	mov    %rcx,%r8
   0x0000000000400789 <+73>:	89 d1	mov    %edx,%ecx
   0x000000000040078b <+75>:	49 39 f1	cmp    %rsi,%r9
   0x000000000040078e <+78>:	72 d0	jb     0x400760 <count_pairs+32>

20	  }
21	  return total;
   0x0000000000400790 <+80>:	c5 f8 77	vzeroupper 
   0x0000000000400793 <+83>:	c3	retq   
   0x0000000000400794 <+84>:	31 c0	xor    %eax,%eax
   0x0000000000400796 <+86>:	c3	retq   
End of assembler dump.
```

So in total in this version manual vectorization was very worth it since it achieved almost 10x speedup over what the compiler was able to achieve.

In general whenever you think to manually vectorize it is worth it to way the extra cost in implementation, maintainability, and generality, over the speed performance that you can get.

# Summary 

- vectorization can help us speed up our code
- the compiler can help us with many forms of vectorization
- when the compiler can't help us we can do it manually using intrinsics



# More Bit and Byte Level Parallelism

## Compression

Let's say we want to store an array of integers using less space by compressing them.

Most commonly used integers use less than the full 32 bits, so we may be able to store them much smaller.

For example instead of storing full integers we can instead use what is known as run length encoding

A standard integer is stored as follow (I will be using little endian for examples)

![Little Endian](Lec09_source/little_endian.JPG "Little Endian")

Instead of always storing all 4 bytes, we can instead use the bottom 7 bits of each bytes to store actual data, then the top bit of each byte to store if we are done with the integer or need to keep reading.

This way if the integer can be represented in less than 21 bits, we are able to use 3 bytes or less and save space.  This method does take more space if the integer needs more than 28 bits for its data.

Another thing we can do is sort the numbers, and only store the differences instead of the actual numbers to make the average size of the elements we are storing smaller. 

Let's look at some code for how we can decode an element
At the end we want what the difference we found was, and how what the difference we found was

```c++
std::pair<int, int> Decode(const uint8_t *loc) {
  // first check the case if we only use 1 byte by checking the top bit of the first byte
  // this is done becuase we need to special case what happens if we are 0
  if ((*loc & 0x80UL) == 0) {
    old_size = *loc > 0;
    return {*loc, old_size};
  }
  uint64_t difference = *loc & 0x7FU;
  uint64_t old_size = 1;
  uint32_t shift_amount = 7;
  // loop over each byte and shift the bits to the correct position
  do {
    loc += 1;
    difference = difference | ((*loc & 0x7FUL) << shift_amount);
    old_size += 1;
    shift_amount += 7;
  } while (*loc & 0x80U);
  return {difference, old_size};
}
```

What if instead of having to read these bytes in 1 at a time we could read them all in parallel

We will use some features from the BMI2 instruction set which allows us to do parallel operations on all bits in a word

We will use two different instructions to help us.

The first is `_pext_u64` which takes in a 64 bit integer, and a 64 bit pattern, it extracts the specified bits from the integer and puts them in continuous low bits of the output.

The second is `__tzcnt_u64` which returns the number of trailing zeros in an integer


```c++

static constexpr std::array<uint64_t, 5> extract_masks = {
    0x000000000000007FUL, 0x0000000000007F7FUL, 0x00000000007F7F7FUL,
    0x000000007F7F7F7FUL, 0x0000007F7F7F7F7FUL};


std::pair<int, int> Decode(const uint8_t *loc) {

  // once again special case the first byte
  if ((*loc & 0x80UL) == 0) {
    old_size = *loc > 0;
    return {*loc, old_size};
  }
  // load the next 64 bits of data which is big enough to always contain the next compressed integer
  uint64_t chunks = unaligned_load<uint64_t>(loc);

  // extract the top bit from each byte
  uint64_t mask = _pext_u64(chunks, 0x8080808080808080UL);

  // find the first bit that is 0 which corosponds to how long the compressed integer is
  int32_t index = __tzcnt_u64(~mask);

  // extract the data bits 
  uint64_t difference = _pext_u64(chunks, extract_masks[index]);

  return {difference, index + 1};

}
```


We can actually do this slightly better by exploiting some pipeline parallelism

```c++

static constexpr std::array<uint64_t, 5> extract_masks2 = {
    0b1111111UL,
    0b11111111111111UL,
    0b111111111111111111111UL,
    0b1111111111111111111111111111UL,
    0b11111111111111111111111111111111111ULL};


std::pair<int, int> Decode(const uint8_t *loc) {

  // once again special case the first byte
  if ((*loc & 0x80UL) == 0) {
    old_size = *loc > 0;
    return {*loc, old_size};
  }
  // load the next 64 bits of data which is big enough to always contain the next compressed integer
  uint64_t chunks = unaligned_load<uint64_t>(loc);

  // extract the top bit from each byte
  uint64_t mask = _pext_u64(chunks, 0x8080808080808080UL);

  // find the first bit that is 0 which corosponds to how long the compressed integer is
  int32_t index = __tzcnt_u64(~mask);

  // extract out the low 7 bits from each byte
  // this operation is no longer dependant on previous __tzcnt_u64, so both can happen in parallel at a hardware level
  uint64_t data_bits = _pext_u64(chunks, 0x7F7F7F7F7F7F7F7FUL);

  // mask out the data you don't care about, notice the masks are different since it is after the extract and pack
  uint64_t difference = data_bits & extract_masks2[index];

  return {difference, index + 1};

}
```
